In [21]:
import pandas as pd
import requests

In [2]:
baseurl = 'https://crashviewer.nhtsa.dot.gov/CrashAPI'
baseurl

'https://crashviewer.nhtsa.dot.gov/CrashAPI'

In [ ]:
endpoint = '/crashes/GetCaseList?states=49&fromYear=2014&toYear=2015&minNumOfVehicles=1&maxNumOfVehicles=6&format=json'

url = baseurl + endpoint

In [4]:
r = requests.get(url)
r.status_code

200

In [6]:
crash_list = pd.json_normalize(r.json()['Results'][0])


In [17]:
df = []
cases = crash_list['St_Case']
for case in cases:
    end2 = f'/crashes/GetCaseDetails?stateCase={case}&caseYear=2022&state=49&format=json'
    r2 = requests.get(baseurl + end2)
    if r2.ok:
        try: 
            temp = pd.json_normalize(r2.json()['Results'][0][0])
            df.append(temp)
        except:
            pass

In [18]:
all_df = pd.concat(df)

In [47]:
df_vars = all_df[['CrashResultSet.STATENAME','CrashResultSet.CITYNAME', 'CrashResultSet.COUNTYNAME', 'CrashResultSet.CaseYear',
                  'CrashResultSet.DAY_WEEKNAME', 'CrashResultSet.HOSP_HRNAME', 'CrashResultSet.LGT_CONDNAME', 'CrashResultSet.RUR_URBNAME',
                  'CrashResultSet.WEATHERNAME', 'CrashResultSet.FATALS']]

In [78]:
accident_data = df_vars.rename(columns = {'CrashResultSet.STATENAME':'state', 'CrashResultSet.CITYNAME':'city', 'CrashResultSet.COUNTYNAME':'county',
                          'CrashResultSet.CaseYear':'year', 'CrashResultSet.DAY_WEEKNAME':'day_of_week', 'CrashResultSet.HOSP_HRNAME':'hospital',
                          'CrashResultSet.LGT_CONDNAME':'lighting', 'CrashResultSet.RUR_URBNAME':'population_type',
                          'CrashResultSet.WEATHERNAME':'weather', 'CrashResultSet.FATALS':'fatalities'})

In [ ]:
accident_data['hospital'] = accident_data['hospital'].apply(lambda x: 0 if x == "Not Applicable (Not Transported) " else 1)

# The hopital variable now represents whether those involved in the accident were transported to the hospital

In [83]:
accident_data.to_csv('accident_data.csv')